In [1]:
import alpaca_trade_api as tradeapi
import requests
from dotenv import load_dotenv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as sco
from pathlib import Path

plt.style.use('fivethirtyeight')
np.random.seed(777)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
load_dotenv(verbose=True)

True

In [3]:
#alpaca_api_key = os.getenv("ALPACA_API_KEY")
#alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca_api_key = "PK2QATFJUTTJTLNCCC9E"
alpaca_secret_key = "OpvkG0g84P35EjDZlJZf0fMdJ4ksJiY6YwxNrN4d"
print(alpaca_api_key)

PK2QATFJUTTJTLNCCC9E


In [4]:
def calculate_sharpe_ratio(df):
    
    df_pct_change = df.pct_change()
    #daily_return
    sharpe_ratio = df_pct_change['close'].mean() / df_pct_change['close'].std()
    
    #annualized sharpe ratio
    return (252 ** 0.5) * sharpe_ratio

In [13]:
csvpath = Path("Resources/forecast_prices.csv")
ETF = pd.read_csv(csvpath)
ETF.head()

,Unnamed: 0,Date,LQD,PZA,USO,TLT,QQQ,VMBS
0,0,2021-01-13 00:00:00-05:00,136.520000,27.090000,35.760000,152.360000,316.000000,54.010000
1,1,2021-01-14 00:00:00-05:00,138.843613,27.096207,34.851509,153.237732,318.907410,53.924221
2,2,2021-01-15 00:00:00-05:00,140.689575,27.091772,34.909908,154.233353,322.679688,53.834557
3,3,2021-01-16 00:00:00-05:00,141.756500,27.087528,34.365986,155.183762,325.903687,53.757324
4,4,2021-01-17 00:00:00-05:00,142.126923,27.084291,34.418289,156.133698,328.474182,53.691902


In [18]:

ETF = ETF.drop(columns = 'Unnamed: 0')
ETF.head()

,LQD,PZA,USO,TLT,QQQ,VMBS
Date,,,,,,
2021-01-13,136.520000,27.090000,35.760000,152.360000,316.000000,54.010000
2021-01-14,138.843613,27.096207,34.851509,153.237732,318.907410,53.924221
2021-01-15,140.689575,27.091772,34.909908,154.233353,322.679688,53.834557
2021-01-16,141.756500,27.087528,34.365986,155.183762,325.903687,53.757324
2021-01-17,142.126923,27.084291,34.418289,156.133698,328.474182,53.691902


In [19]:
sharpe_ratio_list = []
# for each of the tickers, we will calculate the sharpe ratio and store it in a list of dictionaries 
for ticker in tickers:
    sharpe_ratio = calculate_sharpe_ratio(df_ETFs[ticker])
    sharpe_ratio_list.append({"ticker": ticker, "sharpe_ratio" : sharpe_ratio})

# create a data frame with from the list
df_sharpe_ratio= pd.DataFrame(sharpe_ratio_list)
#df_sharpe_ratio

In [26]:
# calculate optimal distribution of etf using efficient frontier and scipy's optimize function
# https://towardsdatascience.com/efficient-frontier-portfolio-optimisation-in-python-e7844051e7f
returns = ETF.pct_change()
mean_returns = returns.mean()
cov_matrix = returns.cov()
num_portfolios = 25000
risk_free_rate = 0

In [33]:
# functions to calculate efficient frontier analysis
# https://towardsdatascience.com/efficient-frontier-portfolio-optimisation-in-python-e7844051e7f
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns
  
def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((6,num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(6)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_var

def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)

    return result

def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    def portfolio_return(weights):
        return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[1]

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

In [38]:
# display function
# https://towardsdatascience.com/efficient-frontier-portfolio-optimisation-in-python-e7844051e7f
def display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, _ = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'], mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=df_closing.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T

    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=df_closing.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print("-"*80)
    print("Maximum Sharpe Ratio Portfolio Allocation\n")
    print("Annualised Return:", round(rp,2))
    print("Annualised Volatility:", round(sdp,2))
    print("\n")
    print(max_sharpe_allocation)
    print("-"*80)
    print("Minimum Volatility Portfolio Allocation\n")
    print("Annualised Return:", round(rp_min,2))
    print("Annualised Volatility:", round(sdp_min,2))
    print("\n")
    print(min_vol_allocation)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')

    target = np.linspace(rp_min, 0.32, 50)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    plt.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
    plt.title('Calculated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)

In [39]:
display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)

ValueError: Shape of passed values is (6, 1), indices imply (7, 1)